## First, try gradient boosting without magic feature

In [6]:
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings('ignore')
%run Functions.py

In [7]:
train = pd.read_csv('data/train_c.csv')
test = pd.read_csv('data/test_c.csv')

In [11]:
train,test = get_additional_features(train,test)
col = test.columns

In [12]:
gb1 = GradientBoostingRegressor(n_estimators=1000,max_features=0.95,learning_rate=0.005,max_depth=4)

In [13]:
gb1_train,gb1_test,ID = get_sklearn_stack_data(gb1,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is -1.12145401894
RMSE of Fold 1 is 17.9983582514
Training 2 Fold
R2 Scored of Fold 2 is 0.435685110191
RMSE of Fold 2 is 10.7305840654
Training 3 Fold
R2 Scored of Fold 3 is 0.582815698482
RMSE of Fold 3 is 7.9741359325
Training 4 Fold
R2 Scored of Fold 4 is 0.52912272281
RMSE of Fold 4 is 8.64138807407
Training 5 Fold
R2 Scored of Fold 5 is 0.0137256337587
RMSE of Fold 5 is 11.4087577269
Start Training
Calculating In-Bag R2 Score
0.674962280129
Calculating Out-Bag R2 Score
0.0879790292594
Calculating In-Bag RMSE
7.22792014518
Calculating Out-Bag RMSE
11.35064481


In [14]:
gb1_test.to_csv('GBDT_1000Rounds_WithoutMagic_For_Comparison.csv',index=False)

In [37]:
save_results(gb1_train,gb1_test,'GBDT_1000Rounds_WithoutMagic_For_Comparison.csv')

## Try: Lasso Without magic feature

In [15]:
from sklearn.linear_model import Lasso

In [16]:
las1 = Lasso(alpha=5,random_state=42)

In [17]:
las1_train,las1_test,ID = get_sklearn_stack_data(las1,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is 0.143337224026
RMSE of Fold 1 is 11.4372345772
Training 2 Fold
R2 Scored of Fold 2 is 0.0879449830023
RMSE of Fold 2 is 13.6418454416
Training 3 Fold
R2 Scored of Fold 3 is 0.140304204579
RMSE of Fold 3 is 11.4470098666
Training 4 Fold
R2 Scored of Fold 4 is 0.141085302528
RMSE of Fold 4 is 11.6709083767
Training 5 Fold
R2 Scored of Fold 5 is 0.154912322122
RMSE of Fold 5 is 10.5606431905
Start Training
Calculating In-Bag R2 Score
0.145449460807
Calculating Out-Bag R2 Score
0.133516807251
Calculating In-Bag RMSE
11.7196688555
Calculating Out-Bag RMSE
11.7515282905


In [18]:
las1_test.to_csv('Lasso_alpha5_without_magic_test.csv',index=False)

**Horrible results**

## lightgbm

In [19]:
import lightgbm as lgb

In [20]:
params = {
        'objective': 'regression',
            'metric': 'rmse',
            'boosting': 'gbdt',
            'learning_rate': 0.0045 , #small learn rate, large number of iterations
            'verbose': 0,
            'num_iterations': 550,
            'bagging_fraction': 0.95,
            'bagging_freq': 1,
            'bagging_seed': 42,
            'feature_fraction': 0.95,
            'feature_fraction_seed': 42,
            'max_bin': 100,
            'max_depth': 4,
            'num_rounds': 1200
        }

In [21]:
lgb_train,lgb_test,ID = get_lgb_stack_data(params,1200,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is 0.405331375509
RMSE of Fold 1 is 9.52913748746
Training 2 Fold
R2 Scored of Fold 2 is 0.439583033111
RMSE of Fold 2 is 10.6934598734
Training 3 Fold
R2 Scored of Fold 3 is 0.594790131372
RMSE of Fold 3 is 7.85886199951
Training 4 Fold
R2 Scored of Fold 4 is 0.537890382543
RMSE of Fold 4 is 8.56055941872
Training 5 Fold
R2 Scored of Fold 5 is 0.661403965389
RMSE of Fold 5 is 6.68467686777
Start Training
Calculating In-Bag R2 Score
0.626576449225
Calculating Out-Bag R2 Score
0.527799777585
Calculating In-Bag RMSE
7.74724546613
Calculating Out-Bag RMSE
8.66533912938


In [35]:
lgb_train.to_csv('StackingData/StackingInput_lgb_without_magic.csv',index=False)

In [36]:
lgb_test.to_csv('submission/sublgb_without_magic.csv',index=False)

## Average two GBDT Sub

In [23]:
gbdt_1 = pd.read_csv('submission/subGBDT_1000trees_depth4_0.55_0.69.csv')

In [24]:
Average = 0.5*gb1_test['y'] + 0.5*gbdt_1['y']

In [27]:
sub=pd.DataFrame({"ID":test['ID'],'y':Average})
sub.to_csv('Average_two_GBDT.csv',index=False)

## OK, start today's main submission

## Trial 1: Two xgb: With Magic and Without Magic

In [29]:
xgb_nomagic = pd.read_csv('submission/subStackThenAveragedXGB.csv')

In [30]:
xgb_withmagic = pd.read_csv('submission/subxgb_depth4_581Rounds_All_Decomp_magic_features_0.5526_0.6378.csv')

In [33]:
Average = xgb_nomagic['y']*0.75+xgb_withmagic['y']*0.25

In [34]:
sub=pd.DataFrame({"ID":test['ID'],'y':Average})
sub.to_csv('submission/Average_two_xgb_magic_nomagic_0.75_0.25.csv',index=False)

## Trial 2: Average xgb,GBDT,lightGBM on the second level

In [38]:
Average = 0.7*xgb_nomagic['y'] + 0.15*lgb_test['y'] + 0.15*gb1_test['y']

In [39]:
sub=pd.DataFrame({"ID":test['ID'],'y':Average})
sub.to_csv('submission/Average_non_magic_xgblgbgbdt_0.7_0.15_0.15.csv',index=False)

## Trial 3: Stacking xgb, GBDT and LightGBM with Lasso

In [41]:
lgb_train,lgb_test = read_data('sublgb_without_magic.csv')

In [43]:
gbdt_train,gbdt_test = read_data('subGBDT_1000Rounds_WithoutMagic_For_Comparison.csv')

In [45]:
y_mean = np.mean(train.y)

In [46]:
xgb_params = {
    'n_trees': 520, 
    'eta': 0.0045,
    'max_depth': 4,
    'subsample': 0.93,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}

In [49]:
#def get_xgb_stack_data(params,rounds,train,col,label,test):
xgb_train,xgb_test,ID = get_xgb_stack_data(xgb_params,1250,train,col,train.y,test)

Training 1 Fold
R2 Scored of Fold 1 is 0.135929170964
RMSE of Fold 1 is 11.4865802816
Training 2 Fold
R2 Scored of Fold 2 is 0.437593717334
RMSE of Fold 2 is 10.7124223828
Training 3 Fold
R2 Scored of Fold 3 is 0.586799917984
RMSE of Fold 3 is 7.93596703829
Training 4 Fold
R2 Scored of Fold 4 is 0.540938596045
RMSE of Fold 4 is 8.53227869704
Training 5 Fold
R2 Scored of Fold 5 is 0.446273561585
RMSE of Fold 5 is 8.54844486172
Start Training
Calculating In-Bag R2 Score
0.637114103361
Calculating Out-Bag R2 Score
0.429506992783
Calculating In-Bag RMSE
7.63715332044
Calculating Out-Bag RMSE
9.44313865229


In [50]:
save_results(xgb_train,xgb_test,'StackThenAveragedXGB.csv')

## Level2

In [52]:
stack_train = pd.DataFrame()
stack_train['ID'] = xgb_train['ID']
stack_train['y'] = xgb_train['y']
stack_train['xgb'] = xgb_train['predicted']
lgb_train = lgb_train[['ID','predicted']]
lgb_train.columns = ['ID','lgb']
stack_train = stack_train.merge(lgb_train,on='ID',how='left')
gbdt_train = gbdt_train[['ID','predicted']]
gbdt_train.columns = ['ID','gbdt']
stack_train = stack_train.merge(gbdt_train,on='ID',how='left')

In [54]:
stack_test = xgb_test
stack_test.columns = ['ID','xgb']
stack_test['lgb']=lgb_test['y']
stack_test['gbdt'] = gbdt_test['y']

In [62]:
las = Lasso(alpha=0.5,random_state=42)

In [63]:
col = stack_test.columns

In [74]:
strain,stest,ID = get_sklearn_stack_data(las,stack_train,col,stack_train['y'],stack_test)

Training 1 Fold
R2 Scored of Fold 1 is 0.32908690313
RMSE of Fold 1 is 10.1216010519
Training 2 Fold
R2 Scored of Fold 2 is 0.413296252623
RMSE of Fold 2 is 10.9413783866
Training 3 Fold
R2 Scored of Fold 3 is 0.594684838112
RMSE of Fold 3 is 7.85988299075
Training 4 Fold
R2 Scored of Fold 4 is 0.536576580585
RMSE of Fold 4 is 8.57271984212
Training 5 Fold
R2 Scored of Fold 5 is 0.628575981962
RMSE of Fold 5 is 7.00123212277
Start Training
Calculating In-Bag R2 Score
0.545908786038
Calculating Out-Bag R2 Score
0.500444111282
Calculating In-Bag RMSE
8.54314964891
Calculating Out-Bag RMSE
8.89936287883


## xgb?

In [72]:
params = {
    'eta': 0.005,
    'max_depth': 2,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}

In [73]:
strain,stest,ID = get_xgb_stack_data(params,1250,stack_train,col,stack_train['y'],stack_test)

Training 1 Fold
R2 Scored of Fold 1 is 0.24648505742
RMSE of Fold 1 is 10.7265982488
Training 2 Fold
R2 Scored of Fold 2 is 0.43205677416
RMSE of Fold 2 is 10.7650256359
Training 3 Fold
R2 Scored of Fold 3 is 0.600678809385
RMSE of Fold 3 is 7.8015488906
Training 4 Fold
R2 Scored of Fold 4 is 0.509076356415
RMSE of Fold 4 is 8.82341318353
Training 5 Fold
R2 Scored of Fold 5 is 0.599594786673
RMSE of Fold 5 is 7.2692456713
Start Training
Calculating In-Bag R2 Score
0.569639301336
Calculating Out-Bag R2 Score
0.477578356811
Calculating In-Bag RMSE
8.31692501831
Calculating Out-Bag RMSE
9.07716632602


In [75]:
save_results(strain,stest,'stack_xgb_lgb_gbdt_by_las.csv')

## Average Results--- Last Try Today

In [76]:
input1 = pd.read_csv('submission/Repro_Ker_2.csv')

In [77]:
Average = 0.7*input1.y + 0.15*lgb_test['y'] + 0.15*gb1_test['y']

In [78]:
sub=pd.DataFrame({"ID":test['ID'],'y':Average})
sub.to_csv('submission/Average_LB_Magic_lgbgbdt_0.7_0.15_0.15.csv',index=False)